In [1]:
from xrpl.account import get_next_valid_seq_number, get_balance
from xrpl.clients import JsonRpcClient
from xrpl.models import Payment, Tx
from xrpl.transaction import sign, submit_and_wait
from xrpl.wallet import Wallet, generate_faucet_wallet
from xrpl.ledger import get_latest_validated_ledger_sequence
import nest_asyncio
from utils import print_balances

In [ ]:
# Create a client to connect to the test network
client = JsonRpcClient("https://s.altnet.rippletest.net:51234")
nest_asyncio.apply()

seed1 = "sEdTo9HXziAK9AUQaxX8jd5Gfv1mqfm"
seed2 = "sEdSPkYBWvGt5crKm8EmugCjtvRNYYM"

# Create two wallets to send money between on the test network
if seed1:
    wallet1 = Wallet.from_seed(seed1)
else:
    wallet1 = generate_faucet_wallet(client, debug=True)

print("Generated wallet 1 with seed: ", wallet1.seed)
print("Address: ", wallet1.classic_address)

if seed2:
    wallet2 = Wallet.from_seed(seed2)
else:
    wallet2 = generate_faucet_wallet(client, debug=True)

print("Generated wallet 2 with seed: ", wallet2.seed)
print("Address: ", wallet2.classic_address)

Generated wallet 1 with seed:  sEdTo9HXziAK9AUQaxX8jd5Gfv1mqfm
Address:  rwiXF251eraXVQgkyEMnja3egbp9uKPvTN
Generated wallet 2 with seed:  sEdSPkYBWvGt5crKm8EmugCjtvRNYYM
Address:  rGtgSL1C9RT5yZwvRZwszqsrFF3JHEUHQ7


In [8]:
# Both balances should be zero since nothing has been sent yet
print("Balances of wallets before Payment tx")
print_balances([wallet1, wallet2], client)

# Create a Payment transaction from wallet1 to wallet2
payment_tx = Payment(
    account=wallet1.address,
    amount="1000",
    destination=wallet2.address,
    fee="1000"
)

# Submit the payment to the network and wait to see a response
#   Behind the scenes, this fills in fields which can be looked up automatically like the fee.
#   It also signs the transaction with wallet1 to prove you own the account you're paying from.
payment_response = submit_and_wait(payment_tx, client, wallet1)
print("Transaction was submitted")

# Create a "Tx" request to look up the transaction on the ledger
tx_response = client.request(Tx(transaction=payment_response.result["hash"]))

# Check whether the transaction was actually validated on ledger
print("Validated:", tx_response.result["validated"])

# Check balances after 1000 drops (.001 XRP) was sent from wallet1 to wallet2
print("Balances of wallets after Payment tx:")
print_balances([wallet1, wallet2], client)

Balances of wallets before Payment tx
Balances:
rwiXF251eraXVQgkyEMnja3egbp9uKPvTN: 100000000
rGtgSL1C9RT5yZwvRZwszqsrFF3JHEUHQ7: 100000000
Transaction was submitted
Validated: True
Balances of wallets after Payment tx:
Balances:
rwiXF251eraXVQgkyEMnja3egbp9uKPvTN: 99998000
rGtgSL1C9RT5yZwvRZwszqsrFF3JHEUHQ7: 100001000


In [4]:
current_validated_ledger = get_latest_validated_ledger_sequence(client)

# prepare the transaction
# the amount is expressed in drops, not XRP
# see https://xrpl.org/basic-data-types.html#specifying-currency-amounts
my_tx_payment = Payment(
    account=wallet1.address,
    amount="2000",
    destination=wallet2.address,
    last_ledger_sequence=current_validated_ledger + 20,
    sequence=get_next_valid_seq_number(wallet1.address, client),
    fee="10",
)
# sign the transaction
my_tx_payment_signed = sign(my_tx_payment, wallet1)

# submit the transaction
tx_response = submit_and_wait(my_tx_payment_signed, client)

print_balances([wallet1, wallet2], client)

Balances:
rhV4A7VVVtHF9cut88JcXWkRzoFZNaEgaw: 99997990
rwFYk8cxtPHaeXwYP1yRqz1kj4rWFGW9h2: 100002000
